In [66]:
from dlisio import dlis
import numpy as np
import pandas as pd
# from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

pd.set_option('display.max_rows', 50)

# extracting lab 

In [ ]:

# loading lab_data from an Excel file

sheet_name = 'HRDH_1111'
lab_data = pd.read_excel("./HRDH_LAB_DATA.xlsx", sheet_name=sheet_name, index_col='Depth_ft')
lab_data


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
# convert data types to float
lab_data = lab_data.apply(pd.to_numeric, errors='coerce')


print("\n📋 DATAFRAME STRUCTURE:")
print(f"Index: {lab_data.index.name} (shape: {lab_data.index.shape})")
print(f"Columns: {list(lab_data.columns)} (shape: {lab_data.shape},)")
print(f"Data types:\n{lab_data.info()}\n")


lab_data


📋 DATAFRAME STRUCTURE:
Index: Depth_ft (shape: (24,))
Columns: ['Sample_ID', 'XRD_Calcite', 'XRD_Quartz', 'XRF_Si', 'XRF_Ca'] (shape: (24, 5),)
<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 10843.0 to 11575.0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sample_ID    24 non-null     int64  
 1   XRD_Calcite  24 non-null     float64
 2   XRD_Quartz   24 non-null     float64
 3   XRF_Si       24 non-null     float64
 4   XRF_Ca       24 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 1.1 KB
Data types:
None



# extracting log dlis

In [ ]:

def dlis_to_df(path, needed=None):
    """Load DLIS file and return a DataFrame with well log data."""
    log = dlis.load(path)[0]
    frame = log.frames[0]
    
    # Get channel names for reference
    channel_names = [ch.name for ch in frame.channels]
    print(f"Found {len(channel_names)} channels: {channel_names[:]}...")
    
    try:
        # Get the curves data as a structured array
        curves_data = frame.curves()
        print(f"Curves data type: {type(curves_data)}")
        
        # Check if it's a structured array with field names
        if hasattr(curves_data.dtype, 'names') and curves_data.dtype.names:
            field_names = curves_data.dtype.names
            print(f"Found structured array with {len(field_names)} fields")
            
            # Create a dictionary to map field names to simple channel names
            field_to_channel = {}
            for field in field_names:
                # For complex field names like (('T.CHANNEL-I.GR-O.1-C.0', 'GR'), '<f4')
                try:
                    if isinstance(field, tuple) and len(field) > 1:
                        if isinstance(field[0], tuple):
                            simple_name = field[0][1]  # Extract 'GR' from the tuple
                            field_to_channel[field] = simple_name
                        continue
                except:
                    pass
                
                # For simpler string field names
                for name in channel_names:
                    if name in str(field):
                        field_to_channel[field] = name
                        break
            
            # Extract data for each field - handle multi-dimensional arrays
            data_dict = {}
            for field in field_names:
                # Get the simple name if we mapped it
                simple_name = field_to_channel.get(field)
                if not simple_name:
                    continue
                
                try:
                    # Get the data
                    array = curves_data[field]
                    
                    # Check if it's multi-dimensional
                    if array.ndim > 1 or (hasattr(array, 'shape') and len(array.shape) > 1):
                        # For multi-dimensional arrays, take first element or flatten
                        print(f"Skipping multi-dimensional array for {simple_name} with shape {array.shape}")
                        continue
                    
                    # Only add 1D arrays to our dictionary
                    data_dict[simple_name] = array
                    
                except Exception as e:
                    print(f"Error extracting {simple_name} from {field}: {e}")
            
            # Find the depth channel
            depth_channel = 'TDEP'
            if depth_channel not in data_dict:
                for name in ['DEPTH', 'DEPT', 'Depth', 'depth']:
                    if name in data_dict:
                        depth_channel = name
                        break
            
            # Create DataFrame
            if data_dict:
                # Make sure all arrays have the same length
                lengths = {name: len(arr) for name, arr in data_dict.items()}
                if len(set(lengths.values())) > 1:
                    print(f"Warning: Arrays have different lengths: {lengths}")
                    # Find minimum length and truncate all arrays
                    min_len = min(lengths.values())
                    data_dict = {name: arr[:min_len] for name, arr in data_dict.items()}
                
                df = pd.DataFrame(data_dict)
                
                # Set depth as index if it exists
                if depth_channel in df.columns:
                    df = df.set_index(depth_channel)
                
                # Filter columns if needed
                if needed is not None:
                    valid_cols = [col for col in needed if col in df.columns]
                    if valid_cols:
                        df = df[valid_cols]
                    else:
                        print(f"Warning: None of the requested columns found in {list(df.columns)}")
                
                print(f"Final DataFrame shape: {df.shape}")
                return df
            else:
                print("Could not extract any data from fields")
                return pd.DataFrame()
        else:
            print("Curves data is not a structured array with named fields")
            # Print more info about the array to help debugging
            print(f"Array shape: {curves_data.shape}")
            print(f"Array dtype: {curves_data.dtype}")
            print(f"First few values: {curves_data[:5]}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"Error processing DLIS file: {e}")
        import traceback

# Path to the DLIS log file for HRDH-1119
LOG_PATH_1119_0 = r"\\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH_1119_0\8375\WL\Field\Deliverables\HRDH_1119_0_29MAY2012_ZDL-CN-DSL_ML_BA.dlis"

# Load log data
log_df = dlis_to_df(path=LOG_PATH_1119_0)

log_df

,BIT,BVOL,CAL,CHT,CN,CVOL,GR,HRD1,HRD2,K,...,SPD,SSD,SSN,TEN,TH,U,WTBH,ZCOR,ZDEN,ZDNC
TDEP,,,,,,,,,,,,,,,,,,,,,
10842.75,-999.250,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,226.94301,-999.25,-999.25,-999.25
10843.00,-999.250,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,226.97990,-999.25,-999.25,-999.25
10843.25,-999.250,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,226.99045,-999.25,-999.25,-999.25
10843.50,-999.250,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,226.98154,-999.25,-999.25,-999.25
10843.75,-999.250,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,226.96066,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11579.50,8.375,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25000,-999.25,-999.25,-999.25
11579.75,8.375,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25000,-999.25,-999.25,-999.25
11580.00,8.375,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,...,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25000,-999.25,-999.25,-999.25


# cleaning log

In [ ]:

# -999.25 are usually null values inside dlis files
null_values = -999.25

# Replace -999.25 with NaN FIRST
log_df_clean = log_df.replace(null_values, np.nan)
log_df_clean




,BIT,BVOL,CAL,CHT,CN,CVOL,GR,HRD1,HRD2,K,...,SPD,SSD,SSN,TEN,TH,U,WTBH,ZCOR,ZDEN,ZDNC
TDEP,,,,,,,,,,,,,,,,,,,,,
10842.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,226.94301,NaN,NaN,NaN
10843.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,226.97990,NaN,NaN,NaN
10843.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,226.99045,NaN,NaN,NaN
10843.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,226.98154,NaN,NaN,NaN
10843.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,226.96066,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11579.50,8.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11579.75,8.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11580.00,8.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# DEPTH ANALYSIS: Compare sampling characteristics between log and lab data
print("=" * 50)

print("Log depths summary:")
print(f"Count: {len(log_df.index)}")  
print(f"Range: {log_df.index.min():.1f} - {log_df.index.max():.1f} ft\n")  
log_step = np.diff(log_df.index).mean()  
print(f"Depth step (mean): {log_step:.2f} ft \nMin Step: {np.diff(log_df.index).min():.2f} \nMax Step: {np.diff(log_df.index).max():.2f}")  


print("=" * 50)


print("Lab depths summary:")
print(f"Count: {len(lab_data.index)}") 
print(f"Range: {lab_data.index.min():.1f} - {lab_data.index.max():.1f} ft\n")  
lab_step = np.diff(lab_data.index).mean()  
print(f"Depth step (mean): {lab_step:.2f} ft \nMin Step: {np.diff(lab_data.index).min():.2f} \nMax Step: {np.diff(lab_data.index).max():.2f}")  


Log depths summary:
Count: 2952
Range: 10842.75 - 11580.5 ft
Log depth step (mean): 0.25 ft

Lab depths summary:
Count: 24
Range: 10843.0 - 11575.0 ft
Lab depth step (mean): 31.83 ft

Detailed step analysis:
Log steps - Min: 0.25, Max: 0.25
Lab steps - Min: 7.25, Max: 100.25


# MISSING DATA SUMMARY

In [ ]:
#Missing Data Summary for log data
# Calculate missingness summary
null_pct = log_df_clean.isna().mean() * 100

# Show all columns with their percentages
missing_cols = null_pct.sort_values(ascending=False)

# Show aggregate summary statistics for many columns
print("🔍 MISSING DATA SUMMARY:")
print("=" * 50)
print(f"Log DATASET: {log_df_clean.shape[0]} samples × {log_df_clean.shape[1]} curves")

print(f"Total columns: {len(null_pct)}")
print(f"Columns with missing data: {len(missing_cols)}")
print(f"Max missing %: {null_pct.max():.1f}%")
print(f"Avg missing %: {null_pct.mean():.1f}%")

# Quick check for duplicates
# print("\n🔍 DUPLICATE DEPTH CHECK:")
# print(f"Log duplicates: {log_df_clean.index.duplicated().sum()}")
# print(f"Lab duplicates: {lab_data.index.duplicated().sum()}")

# # Show unique vs total counts
# print(f"\nLog depths - Total: {len(log_df_clean)}, Unique: {log_df_clean.index.nunique()}")
# print(f"Lab depths - Total: {len(lab_data)}, Unique: {lab_data.index.nunique()}")

print("=" * 50)


# Show detailed breakdown for all columns
for col, pct in missing_cols.items():
    count = log_df_clean[col].isna().sum()
    print(f"• {col:<20}: {pct:>6.1f}% ({count} missing)")




🔍 MISSING DATA SUMMARY:
📊 Columns with missing data (27/27):

📈 SUMMARY:
• Total columns: 27
• Columns with missing data: 27
• Max missing %: 6.5%
• Avg missing %: 5.1%
•  CN                 :    6.5% (191 missing)
•  ZCOR               :    5.4% (160 missing)
•  ZDNC               :    5.4% (160 missing)
•  ZDEN               :    5.4% (160 missing)
•  RTHU               :    5.4% (159 missing)
•  TH                 :    5.4% (159 missing)
•  U                  :    5.4% (159 missing)
•  K                  :    5.4% (159 missing)
•  PE                 :    5.3% (156 missing)
•  HRD2               :    5.2% (154 missing)
•  HRD1               :    5.2% (154 missing)
•  SHR                :    5.2% (154 missing)
•  SFT2               :    5.2% (154 missing)
•  CVOL               :    5.2% (153 missing)
•  BVOL               :    5.2% (153 missing)
•  SSN                :    5.0% (148 missing)
•  LSN                :    4.9% (144 missing)
•  SSD                :    4.9% (144 missing)
•  

# OVERLAP

In [ ]:
# OVERLAP ZONE ANALYSIS: Identify common depth intervals for data integration
print("🔍 DETAILED OVERLAP ZONE ANALYSIS:")
print("=" * 50)

# Dataset coverage comparison
print(f"📏 Depth Ranges:")
print(f"Log Range: {log_df.index.min():.1f} - {log_df.index.max():.1f} ft")  
print(f"Lab Range: {lab_data.index.min():.1f} - {lab_data.index.max():.1f} ft")  

# Calculate overlap zone boundaries
overlap_start = max(lab_data.index.min(), log_df_clean.index.min())
overlap_end = min(lab_data.index.max(), log_df_clean.index.max())
overlap_span = overlap_end - overlap_start

print("=" * 50)

print(f"Overlap Range: {overlap_start:.1f} - {overlap_end:.1f} ft")
print(f"Span of overlap: {overlap_span:.1f} ft")

# Coverage percentages
lab_total_span = lab_data.index.max() - lab_data.index.min()
log_total_span = log_df_clean.index.max() - log_df_clean.index.min()
overlap_lab_pct = (overlap_span / lab_total_span) * 100
overlap_log_pct = (overlap_span / log_total_span) * 100

print(f"Coverage: {overlap_lab_pct:.1f}% of lab data, {overlap_log_pct:.1f}% of log data")
print("=" * 50)

# Get samples in overlap zone
log_overlap = log_df_clean[(log_df_clean.index >= overlap_start) & (log_df_clean.index <= overlap_end)]
lab_overlap = lab_data[(lab_data.index >= overlap_start) & (lab_data.index <= overlap_end)]

# Calculate Step statistics in overlap zone
if len(log_overlap) > 1:
    log_step_overlap = np.diff(log_overlap.index)
    print(f"LOG STEP IN OVERLAP:")
    print(f"Step (mean): {log_step_overlap.mean():.2f} ft")
    print(f"Min: {log_step_overlap.max():.2f} ft \nStep Max: {log_step_overlap.min():.2f}")
    # print(f"Step Std: {log_step_overlap.std():.2f} ft")

if len(lab_overlap) > 1:
    lab_step_overlap = np.diff(lab_overlap.index)
    print(f"\nLAB STEP IN OVERLAP:")
    print(f"Step (mean): {lab_step_overlap.mean():.2f} ft")
    print(f"Min: {lab_step_overlap.max():.2f} ft \nStep Max: {lab_step_overlap.min():.2f}")
    # print(f"Step Std: {lab_step_overlap.std():.2f} ft")

print("=" * 50)

# Show actual depths in overlap zone
print(f"📍 SAMPLE DEPTHS IN OVERLAP ZONE:")
print(f"🔵 Log depths (first/last 5 of {len(log_overlap):,}):")
if len(log_overlap) > 0:
    # Show first 5
    for i, depth in enumerate(log_overlap.index[:5]):
        print(f"   {i+1:2d}. {depth:.2f} ft")
    if len(log_overlap) > 10:
        print("   ...")
        # Show last 5
        for i, depth in enumerate(log_overlap.index[-5:], start=len(log_overlap)-4):
            print(f"  {i:2d}. {depth:.2f} ft")

print(f"🔴 Lab depths (all {len(lab_overlap)}):")
if len(lab_overlap) > 0:
    for i, depth in enumerate(lab_overlap.index):
        print(f"   {i+1:2d}. {depth:.1f} ft")

#display depths in a df
# Display lab_overlap and log_df_clean side by side for the overlap zone

print("Lab Overlap Samples:")
display(lab_overlap)

print("Log Data:")
display(log_df_clean.loc[log_df_clean.index])

# Data quality assessment for overlap zone
# print(f"\n⭐ OVERLAP ZONE ASSESSMENT:")
# if overlap_span >= 100:
#     print(f"✅ Overlap span: EXCELLENT ({overlap_span:.1f} ft)")
# elif overlap_span >= 50:
#     print(f"⚠️ Overlap span: GOOD ({overlap_span:.1f} ft)")
# else:
#     print(f"❌ Overlap span: POOR ({overlap_span:.1f} ft)")

# if len(lab_overlap) >= 5:
#     print(f"✅ Lab samples: SUFFICIENT ({len(lab_overlap)} samples)")
# elif len(lab_overlap) >= 3:
#     print(f"⚠️ Lab samples: LIMITED ({len(lab_overlap)} samples)")
# else:
#     print(f"❌ Lab samples: INSUFFICIENT ({len(lab_overlap)} samples)")

# if len(log_overlap) >= 100:
#     print(f"✅ Log samples: EXCELLENT ({len(log_overlap):,} samples)")
# elif len(log_overlap) >= 50:
#     print(f"⚠️ Log samples: GOOD ({len(log_overlap)} samples)")
# else:
#     print(f"❌ Log samples: POOR ({len(log_overlap)} samples)")

# Recommendations based on overlap analysis
# print(f"\n💡 OVERLAP RECOMMENDATIONS:")
# if overlap_span < 50:
#     print(f"Consider requesting more lab samples in {overlap_start:.0f}-{overlap_end:.0f} ft zone")
# if len(lab_overlap) < 3:
#     print(f"Minimum 3-5 lab samples needed for reliable")
# if overlap_log_pct < 20:
#     print(f"Overlap covers only {overlap_log_pct:.1f}% of log data - limited analysis scope")
# if overlap_lab_pct < 50:
#     print(f"Consider expanding log coverage to match lab sampling range")

# print(f"\n📊 INTEGRATION POTENTIAL:")
# integration_score = (min(overlap_span/100, 1) * 0.4 + 
#                     min(len(lab_overlap)/5, 1) * 0.3 + 
#                     min(len(log_overlap)/100, 1) * 0.3) * 100

# if integration_score >= 80:
#     print(f"🟢 EXCELLENT ({integration_score:.0f}/100) - Ready for comprehensive analysis")
# elif integration_score >= 60:
#     print(f"🟡 GOOD ({integration_score:.0f}/100) - Suitable for targeted analysis")
# else:
#     print(f"🔴 POOR ({integration_score:.0f}/100) - Limited analysis potential")



Depth statistics:
         Log Depths    Lab Depths
count   2952.000000     24.000000
mean   11211.625000  11121.510417
std      213.078331    232.095270
min    10842.750000  10843.000000
25%    11027.187500  10931.687500
50%    11211.625000  11062.875000
75%    11396.062500  11262.812500
max    11580.500000  11575.000000
Statistics Summary of log_df cleaning

🔍 DUPLICATE DEPTH CHECK:
Log duplicates: 0
Lab duplicates: 0

Log depths - Total: 2952, Unique: 2952
Lab depths - Total: 24, Unique: 24


# join

In [ ]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

def match_lab_to_log(log_df, lab_df, tol=0.1):
    """
    For each lab depth, find the nearest log depth within `tol`.
    Returns at most len(lab_df) matched pairs.
    Adds columns for Distance and Match_Type.
    """
    print(f"\n🔧 FUNCTION START - INPUT VALIDATION:")
    print(f"   • Log DataFrame shape: {log_df.shape}")
    print(f"   • Lab DataFrame shape: {lab_df.shape}")
    
    # Ensure unique indices
    log = log_df[~log_df.index.duplicated(keep='first')]
    lab = lab_df[~lab_df.index.duplicated(keep='first')]
    
    print(f"   • After deduplication - Log: {len(log)}, Lab: {len(lab)}")
    
    if len(log) == 0 or len(lab) == 0:
        print("❌ ERROR: Empty datasets after deduplication!")
        return pd.DataFrame()
    
    # Convert to float64 arrays
    log_depths = np.array(log.index.values, dtype=np.float64).reshape(-1, 1)
    lab_depths = np.array(lab.index.values, dtype=np.float64).reshape(-1, 1)
    
    print(f"   • Log depths sample: {log_depths[:3].flatten()}")
    print(f"   • Lab depths sample: {lab_depths[:3].flatten()}")
    
    # Build KD-Tree on log depths
    tree = cKDTree(log_depths)
    
    # Query each lab depth
    dists, idxs = tree.query(lab_depths, distance_upper_bound=tol)
    
    # FIXED: Check for finite distances (successful matches)
    mask = np.isfinite(dists)
    
    print(f"\n🔍 DEBUG INFO:")
    print(f"   • Lab depths range: {lab_depths.min():.2f} - {lab_depths.max():.2f}")
    print(f"   • Log depths range: {log_depths.min():.2f} - {log_depths.max():.2f}")
    print(f"   • Tolerance: {tol} ft")
    print(f"   • Valid matches found: {mask.sum()}/{len(mask)}")
    
    if mask.sum() > 0:
        print(f"   • Min distance: {dists[mask].min():.2f} ft")
        print(f"   • Max distance: {dists[mask].max():.2f} ft")
        
        # DETAILED MATCH VERIFICATION
        print(f"\n🔍 DETAILED MATCH VERIFICATION:")
        for i in range(min(24, mask.sum())):
            match_idx = np.where(mask)[0][i]
            lab_depth = lab_depths[match_idx][0]
            log_idx = idxs[match_idx]
            log_depth = log_depths[log_idx][0]
            distance = dists[match_idx]
            print(f"   Match {i+1}: Lab {lab_depth:.2f} → Log {log_depth:.2f} (Δ{distance:.2f} ft)")
    
    if mask.sum() == 0:
        print("❌ No matches found within tolerance!")
        # Try with larger tolerance for diagnosis
        dists_large, idxs_large = tree.query(lab_depths, distance_upper_bound=1.0)
        mask_large = np.isfinite(dists_large)
        
        if mask_large.sum() > 0:
            print(f"\n🔍 CLOSEST MATCHES (within 1.0 ft):")
            for i in range(min(5, mask_large.sum())):
                match_idx = np.where(mask_large)[0][i]
                lab_depth = lab_depths[match_idx][0]
                log_idx = idxs_large[match_idx]
                log_depth = log_depths[log_idx][0]
                distance = dists_large[match_idx]
                print(f"   Lab {lab_depth:.2f} → Log {log_depth:.2f} (Δ{distance:.2f} ft)")
        
        return pd.DataFrame()
    
    # Get matched samples - FIXED: Only include valid matches
    matched_lab_indices = np.where(mask)[0]
    matched_log_indices = idxs[mask]
    
    matched_lab = lab.iloc[matched_lab_indices].reset_index()
    matched_log = log.iloc[matched_log_indices].reset_index()
    
    print(f"   • Matched samples extracted: {len(matched_lab)} pairs")
    
    # Rename depth columns and add prefixes
    matched_lab = matched_lab.rename(columns={matched_lab.columns[0]: 'Lab_Depth'})
    matched_log = matched_log.rename(columns={matched_log.columns[0]: 'Log_Depth'})
    
    # Add prefixes to avoid column name conflicts
    lab_cols = ['Lab_Depth'] + [f'Lab_{col}' for col in matched_lab.columns[1:]]
    log_cols = ['Log_Depth'] + [f'Log_{col}' for col in matched_log.columns[1:]]
    
    matched_lab.columns = lab_cols
    matched_log.columns = log_cols

    # Concatenate side-by-side
    joined_df = pd.concat([matched_lab, matched_log], axis=1)
    
    # Add distance column - CALCULATE ACTUAL DISTANCES
    joined_df['Distance'] = np.abs(joined_df['Lab_Depth'] - joined_df['Log_Depth'])
    
    # Add match type column
    joined_df['Match_Type'] = np.where(joined_df['Distance'] == 0, 'Exact', 'Near')
    
    # VERIFICATION: Show actual matches
    print(f"\n✅ FINAL VERIFICATION - First 5 matches:")
    for i in range(min(5, len(joined_df))):
        row = joined_df.iloc[i]
        print(f"   Lab: {row['Lab_Depth']:.2f} → Log: {row['Log_Depth']:.2f} (Δ{row['Distance']:.2f} ft)")
    
    return joined_df

# Test with very strict tolerance first
tolerance_ft = 0.01

print("🚀 TESTING WITH STRICT TOLERANCE...")
joined = match_lab_to_log(log_df_clean, lab_overlap, tol=tolerance_ft)

if len(joined) > 0:
    print(f"\n✅ Found {len(joined)} matches with {tolerance_ft} ft tolerance")
    
    # Show statistics
    print(f"\n📊 MATCH STATISTICS:")
    print(f"   • Average distance: {joined['Distance'].mean():.3f} ft")
    print(f"   • Max distance: {joined['Distance'].max():.3f} ft")
    print(f"   • Exact matches: {(joined['Distance'] == 0).sum()}")
    print(f"   • Near matches: {(joined['Distance'] > 0).sum()}")
    
    # Save results for verification
    verification_df = joined[['Lab_Depth', 'Log_Depth', 'Distance']].copy()
    verification_df.to_csv('verification_matches.csv', index=False)
    print(f"\n💾 Saved verification data to 'verification_matches.csv'")
    
else:
    print(f"\n❌ No matches found with {tolerance_ft} ft tolerance")
    print("Trying with larger tolerance...")
    
    # Try with 0.1 ft tolerance
    joined = match_lab_to_log(log_df_clean, lab_overlap, tol=0.01)

Lab samples: 24 → Matched pairs: 24

✅ FINAL RESULTS:
   • Found 24 matches with 1 ft tolerance
   • Lab samples: 24
   • This is ≤ 24 ✓

📊 MATCH BREAKDOWN:
   • Exact matches: 24

🎯 DEPTH ACCURACY:
   • Average distance: 0.00 ft
   • Max distance: 0.00 ft
   • Min distance: 0.00 ft


,Lab_Depth,Sample_ID,XRD_Calcite,XRD_Quartz,XRF_Si,XRF_Ca,Log_Depth,BIT,BVOL,CAL,...,SSN,TEN,TH,U,WTBH,ZCOR,ZDEN,ZDNC,Distance,Match_Type
0,10843.00,1,2.1,55.3,31.2,15.4,10843.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,226.97990,NaN,NaN,NaN,0.0,Exact
1,10850.25,2,1.9,60.2,30.1,14.8,10850.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,227.03395,NaN,NaN,NaN,0.0,Exact
2,10875.50,3,2.5,52.7,32.0,16.1,10875.50,8.375,322.606380,8.884544,...,1399.2987,NaN,0.917431,1.419340,227.40001,NaN,NaN,NaN,0.0,Exact
3,10890.75,4,0.8,45.0,28.5,12.3,10890.75,8.375,316.080200,8.847898,...,1540.8405,5060.7354,0.851867,1.201015,227.33203,-0.886580,2.426006,2.421260,0.0,Exact
4,10905.00,5,1.2,48.5,29.7,13.5,10905.00,8.375,309.983200,8.843657,...,1542.8284,5088.0650,0.939252,0.947135,227.82405,-0.864319,2.411453,2.406701,0.0,Exact
5,10920.25,6,1.5,50.1,30.0,14.0,10920.25,8.375,303.457030,8.875068,...,1517.7223,5084.7440,0.589605,1.447574,227.92833,-0.863133,2.512076,2.507457,0.0,Exact
6,10935.50,7,2.0,53.0,31.5,15.0,10935.50,8.375,296.933070,8.866194,...,1737.4972,5094.7803,0.874296,1.064741,228.42671,-0.841954,2.480267,2.475582,0.0,Exact
7,10960.75,8,1.8,58.4,30.8,14.9,10960.75,8.375,286.112300,8.883152,...,1449.1405,5089.1733,0.999127,1.260219,228.84143,-0.926843,2.184703,2.180397,0.0,Exact
8,10980.00,9,2.3,54.2,31.8,15.2,10980.00,8.375,277.868260,8.837547,...,2147.9320,5105.3510,0.555344,1.537119,229.25688,-0.858118,3.132798,3.133401,0.0,Exact
9,11000.25,10,2.7,57.1,32.5,15.8,11000.25,8.375,269.120900,8.870407,...,1787.5247,5119.4385,0.430017,1.583948,229.17587,-0.795233,2.587357,2.582968,0.0,Exact


In [ ]:
# VERIFICATION: Check if lab depths are exactly on the log grid
print("🔍 VERIFYING DEPTH GRID ALIGNMENT:")
print("=" * 50)

# Check log depth spacing
log_spacing = np.diff(log_df_clean.index)
print(f"Log depth spacing:")
print(f"   • Mean: {log_spacing.mean():.3f} ft")
print(f"   • Min: {log_spacing.min():.3f} ft") 
print(f"   • Max: {log_spacing.max():.3f} ft")
print(f"   • Unique spacings: {np.unique(log_spacing)}")

# Check if lab depths are multiples of 0.25
lab_depths_array = lab_overlap.index.values
log_depths_array = log_df_clean.index.values

print(f"\n🔬 LAB DEPTH ANALYSIS:")
print(f"   • Lab depths modulo 0.25:")
for i, depth in enumerate(lab_depths_array[:10]):
    remainder = depth % 0.25
    print(f"     {depth:.2f} % 0.25 = {remainder:.3f}")

print(f"\n🔬 CHECKING FOR EXACT MATCHES:")
exact_matches = 0
for lab_depth in lab_depths_array:
    if lab_depth in log_depths_array:
        exact_matches += 1
        print(f"   ✅ {lab_depth:.2f} found in log data")
    else:
        closest_log = log_depths_array[np.argmin(np.abs(log_depths_array - lab_depth))]
        distance = abs(lab_depth - closest_log)
        print(f"   ❌ {lab_depth:.2f} NOT found, closest: {closest_log:.2f} (Δ{distance:.3f})")

print(f"\n📊 SUMMARY:")
print(f"   • Exact matches found: {exact_matches}/{len(lab_depths_array)}")
print(f"   • This explains why tolerance of 0.1 ft works!")

# Test with impossible tolerance to see what happens
print(f"\n🧪 TESTING WITH 0.01 ft TOLERANCE:")
tiny_matches = match_lab_to_log(log_df_clean, lab_overlap, tol=0.01)
print(f"   • Matches with 0.01 ft: {len(tiny_matches)}")

print(f"\n💡 CONCLUSION:")
if exact_matches == len(lab_depths_array):
    print(f"   ✅ Lab samples were taken at EXACT log measurement depths")
    print(f"   ✅ Both datasets use the same 0.25 ft depth grid")
    print(f"   ✅ Perfect matches are LEGITIMATE, not a bug!")
else:
    print(f"   ⚠️ Some misalignment exists - investigate further")

In [ ]:
# Test with extremely strict tolerance
print("🧪 TESTING WITH IMPOSSIBLE TOLERANCE (0.001 ft):")
impossible_matches = match_lab_to_log(log_df_clean, lab_overlap, tol=0.001)
print(f"   • Matches with 0.001 ft: {len(impossible_matches)}")

print("\n🧪 TESTING WITH RIDICULOUS TOLERANCE (0.0001 ft):")
ridiculous_matches = match_lab_to_log(log_df_clean, lab_overlap, tol=0.0001)
print(f"   • Matches with 0.0001 ft: {len(ridiculous_matches)}")

print("\n📊 FINAL PROOF:")
print("If all tests return 24 matches, then distances are truly 0.00 ft")
print("This confirms perfect depth alignment - not a bug!")

# DATA INTEGRATION ASSESSMENT

In [ ]:
# COMPREHENSIVE DATA INTEGRATION ASSESSMENT
print("🔍 COMPREHENSIVE DATA INTEGRATION ASSESSMENT")
print("=" * 60)

# 1. Basic Statistics
lab_coverage = (len(joined) / len(lab_data)) * 100
log_coverage = (len(joined) / len(log_df_clean)) * 100

lab_span = lab_data.index.max() - lab_data.index.min()
log_span = log_df_clean.index.max() - log_df_clean.index.min()

matched_span = joined['Lab_Depth'].max() - joined['Lab_Depth'].min()
avg_depth_diff = abs(joined['Log_Depth'] - joined['Lab_Depth']).mean()

print(f"📊 DATA OVERVIEW:")
print(f"   • Lab samples: {len(lab_data)} | Log samples: {len(log_df_clean)}")
print(f"   • Matched pairs: {len(joined)}")
# print(f"   • Data ratio: 1 lab sample per {len(log_df_clean)//len(lab_data)} log samples") not usful

# 2. Coverage Analysis
print(f"\n📈 COVERAGE ANALYSIS:")
print(f"   • Lab utilization: {lab_coverage:.1f}% ({len(joined)}/{len(lab_data)})")
print(f"   • Log coverage: {log_coverage:.1f}% ({len(joined)}/{len(log_df_clean)})")

# 3. Depth Range Analysis
print(f"\n📏 DEPTH RANGE ANALYSIS:")
print(f"   • Lab data span: {lab_span:.1f} ft")
print(f"   • Log data span: {log_span:.1f} ft")
print(f"   • Matched (joined) span: {matched_span:.1f} ft")
print(f"   • Lab covers {(lab_span/log_span)*100:.1f}% of well depth")

# 4. Depth Accuracy
print(f"\n🎯 DEPTH ACCURACY:")
print(f"   • Average depth difference: {avg_depth_diff:.2f} ft")
print(f"   • Max depth difference: {abs(joined['Log_Depth'] - joined['Lab_Depth']).max():.2f} ft")
print(f"   • Min depth difference: {abs(joined['Log_Depth'] - joined['Lab_Depth']).min():.2f} ft")

# 5. Sampling Density
lab_density = len(lab_data) / (lab_span / 100)
log_density = len(log_df_clean) / (log_span / 100)
print(f"\n📍 SAMPLING DENSITY:")
print(f"   • Lab samples: {lab_density:.1f} per 100 ft")
print(f"   • Log samples: {log_density:.1f} per 100 ft")
# This shows why you have poor matching:
print(f"Lab sample spacing: ~{100/14.5:.1f} feet apart")
print(f"Log sample spacing: ~{100/400.1:.2f} feet apart")
print(f"Density ratio: {400.1/14.5:.1f}:1 (log:lab)")

# 6. Overlap Analysis
overlap_start = max(lab_data.index.min(), log_df_clean.index.min())
overlap_end = min(lab_data.index.max(), log_df_clean.index.max())
overlap_span = overlap_end - overlap_start
print(f"\n🔄 OVERLAP ZONE ANALYSIS:")
print(f"Overlap zone: {overlap_start:.1f} - {overlap_end:.1f} ft")

# Get samples in overlap zone - FIXED the bug
log_overlap = joined[(joined.index >= overlap_start) & (joined.index <= overlap_end)]
lab_overlap = lab_data[(lab_data.index >= overlap_start) & (lab_data.index <= overlap_end)]

print(f"Log samples in overlap: {len(log_overlap)}")
print(f"Lab samples in overlap: {len(lab_overlap)}")

# Show first few samples from each in overlap zone
print(f"\n📍 FIRST 10 SAMPLES IN OVERLAP ZONE:")
overlap_comparison = pd.DataFrame({
    'Log_Depths_Overlap': pd.Series(log_overlap.index[:10]),
    'Lab_Depths_Overlap': pd.Series(lab_overlap.index[:10])
})
print(overlap_comparison)

# Method 3: Show depth spacing analysis
print(f"\n📏 DEPTH SPACING ANALYSIS:")
log_spacing = np.diff(log_df_clean.index)
lab_spacing = np.diff(lab_data.index)

spacing_stats = pd.DataFrame({
    'Log_Spacing': [log_spacing.mean(), log_spacing.std(), log_spacing.min(), log_spacing.max()],
    'Lab_Spacing': [lab_spacing.mean(), lab_spacing.std(), lab_spacing.min(), lab_spacing.max()]
}, index=['Mean', 'Std', 'Min', 'Max'])


# 7. Quality Assessment
print(f"\n⭐ QUALITY ASSESSMENT:")
if lab_coverage >= 80:
    print(f"   ✅ Lab utilization: EXCELLENT ({lab_coverage:.1f}%)")
elif lab_coverage >= 60:
    print(f"   ⚠️ Lab utilization: GOOD ({lab_coverage:.1f}%)")
else:
    print(f"   ❌ Lab utilization: POOR ({lab_coverage:.1f}%)")

if log_coverage >= 50:
    print(f"   ✅ Log coverage: EXCELLENT ({log_coverage:.1f}%)")
elif log_coverage >= 20:
    print(f"   ⚠️ Log coverage: FAIR ({log_coverage:.1f}%)")
else:
    print(f"   ❌ Log coverage: POOR ({log_coverage:.1f}%)")

if matched_span >= 100:
    print(f"   ✅ Depth span: EXCELLENT ({matched_span:.1f} ft)")
elif matched_span >= 50:
    print(f"   ⚠️ Depth span: FAIR ({matched_span:.1f} ft)")
else:
    print(f"   ❌ Depth span: POOR ({matched_span:.1f} ft)")

if avg_depth_diff <= 1:
    print(f"   ✅ Depth accuracy: EXCELLENT ({avg_depth_diff:.2f} ft)")
elif avg_depth_diff <= 2:
    print(f"   ⚠️ Depth accuracy: GOOD ({avg_depth_diff:.2f} ft)")
else:
    print(f"   ❌ Depth accuracy: POOR ({avg_depth_diff:.2f} ft)")

# 8. Overall Assessment
excellent_count = sum([lab_coverage >= 80, log_coverage >= 50, matched_span >= 100, avg_depth_diff <= 1])
good_count = sum([60 <= lab_coverage < 80, 20 <= log_coverage < 50, 50 <= matched_span < 100, 1 < avg_depth_diff <= 2])

print(f"\n🏆 OVERALL ASSESSMENT:")
if excellent_count >= 3:
    print(f"   ✅ EXCELLENT - Ready for comprehensive analysis")
elif excellent_count + good_count >= 3:
    print(f"   ⚠️ GOOD - Suitable for limited analysis")
else:
    print(f"   ❌ POOR - Needs improvement before analysis")

# 9. Recommendations
print(f"\n💡 RECOMMENDATIONS:")
if log_coverage < 10:
    print(f"   • Increase tolerance to {tolerance_ft * 2}-{tolerance_ft * 5} ft")
    print(f"   • Check for systematic depth shifts")
    print(f"   • Verify depth reference datums")
if matched_span < 50:
    print(f"   • Request additional lab samples")
    print(f"   • Focus analysis on overlap zone ({overlap_start:.1f}-{overlap_end:.1f} ft)")
if lab_span/log_span < 0.5:
    print(f"   • Consider interpolation for full-well analysis")
    print(f"   • Use matched data for calibration only")

print(f"\n📝 SUMMARY: Lab utilization is {lab_coverage:.1f}%, but only {log_coverage:.1f}% log coverage limits comprehensive analysis.")

🔍 COMPREHENSIVE DATA INTEGRATION ASSESSMENT
📊 DATA OVERVIEW:
   • Lab samples: 24 | Log samples: 2952
   • Matched pairs: 24

📈 COVERAGE ANALYSIS:
   • Lab utilization: 100.0% (24/24)
   • Log coverage: 0.8% (24/2952)

📏 DEPTH RANGE ANALYSIS:
   • Lab data span: 732.0 ft
   • Log data span: 737.8 ft
   • Matched (joined) span: 732.0 ft
   • Lab covers 99.2% of well depth

🎯 DEPTH ACCURACY:
   • Average depth difference: 0.00 ft
   • Max depth difference: 0.00 ft
   • Min depth difference: 0.00 ft

📍 SAMPLING DENSITY:
   • Lab samples: 3.3 per 100 ft
   • Log samples: 400.1 per 100 ft
Lab sample spacing: ~6.9 feet apart
Log sample spacing: ~0.25 feet apart
Density ratio: 27.6:1 (log:lab)

🔄 OVERLAP ZONE ANALYSIS:
Overlap zone: 10843.0 - 11575.0 ft
Log samples in overlap: 0
Lab samples in overlap: 24

📍 FIRST 10 SAMPLES IN OVERLAP ZONE:
   Log_Depths_Overlap  Lab_Depths_Overlap
0                 NaN            10843.00
1                 NaN            10850.25
2                 NaN      

# Visualizations  